<a href="https://colab.research.google.com/github/viral29-ui/.github-workflows-is-website-vulnerable.yml/blob/main/Copy_of_Streamlit_Notebook_DoNotEdit_Student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
!pip -q install streamlit > /dev/null
!pip -q install pyngrok > /dev/null
!wget https://www.dropbox.com/s/072b5vf4b33bu1l/emotion_detection_model_for_streamlit.h5 > /dev/null
!wget https://www.dropbox.com/s/p52z1qle0x1uf6f/happy.png > /dev/null

--2021-06-15 23:33:43--  https://www.dropbox.com/s/072b5vf4b33bu1l/emotion_detection_model_for_streamlit.h5
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/072b5vf4b33bu1l/emotion_detection_model_for_streamlit.h5 [following]
--2021-06-15 23:33:44--  https://www.dropbox.com/s/raw/072b5vf4b33bu1l/emotion_detection_model_for_streamlit.h5
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucfac92616ea4d7925ff7b6c7a60.dl.dropboxusercontent.com/cd/0/inline/BQc4s38FhvZzTQBxatow_IVqZj_V6N1OSH7LEYIBzQNfnTORtOQbDv8SLZOm8BZx3ldGq1y1hd6GjBGeH6ZLm_nCxrRLbFslnHJRBxOfMJGN6cchI3ikEKvQXkA4XosUueIuNfOj6u1lAcH_BeewoJdo/file# [following]
--2021-06-15 23:33:44--  https://ucfac92616ea4d7925ff7b6c7a60.dl.dropboxusercontent.com/cd/0/inline/BQc4s3

In [51]:
import tensorflow as tf
import pandas as pd
import cv2
import plotly.express as px
import numpy as np

# Streamlit - Deploying your model to the web

The goal of this session is to learn how to deploy the models that we have been training to the web so they can be shared with the world!

Let's start with an example. 

Checkout [this example](https://share.streamlit.io/thoppe/alph-the-sacred-river/main) and answer the following questions
**Questions:**
* Who is this application for?
* What are 2 ways the user can input data - are these intuitive ways of interacting with the app?
* What does the application do with the data?
* Evaluate the ease of use and look of the application. 

**Exercise**: As a group brainstorm pick two of your projects and answer the following questions related to deployment and app design
* Who do you want to use your app?
* What does the app do for the user?
* How does the user interact with the app?


As our example today we'll try to deploy an **emotion detection model** to the web. 

**Discuss:** What important applications of emotion detection are there?

**Exercise:** Do the previous exercise for today's example of **emotion detection**.

## Step 1: Making a website with streamlit

Streamlit interprets Python files as a website! This is great for several reasons
* No need to know HTML, CSS, Javascript,... etc
* Easy to use our trained models which are already in Python!

We'll write everything to a file called app.py. Run the next cell to create the file and then double click app.py on the left to see the contents of app.py. 

*If you don't see the file you might need to click the refresh button in the file explorer.*



In [68]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
import numpy as np
import cv2

st.title("Emotion detector")
st.header("Detects your emotions")
st.write("This app detects emotion through the use of machine learning")
model = tf.keras.models.load_model("emotion_detection_model_for_streamlit.h5")
f = st.file_uploader("Upload Image")
if f is not None: 
   file_bytes = np.asarray(bytearray(f.read()), dtype=np.uint8)
   image = cv2.imdecode(file_bytes, 1)
   st.image(image, channels="BGR")
   with open("happy.png", "rb") as f:
     file_bytes = np.asarray(bytearray(f.read()))
     image = cv2.imdecode(file_bytes, 1)

print("Shape of image:")
print(image.shape)
print(f"Maximum value in the image: ")
print(image.max())
print("Minimum value in the image:")
print(image.min())

px.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
resized = cv2.resize(image, (48, 48), interpolation=cv2.INTER_LANCZOS4)
print(resized.shape)
px.imshow(cv2.cvtColor(resized, cv2.COLOR_BGR2RGB))
gray_1d = np.mean(resized, axis=-1)
gray = np.zeros_like(resized)
gray[:,:,0] = gray_1d
gray[:,:,1] = gray_1d
gray[:,:,2] = gray_1d
px.imshow(gray)
normalized = gray/255
px.imshow(normalized)
print("Shape of image:")
print(normalized.shape)
print(f"Maximum value in the image: ")
print(normalized.max())
print("Minimum value in the image:")
print(normalized.min())
EMOTIONS = ['ANGRY', 'HAPPY', 'SAD', 'SURPRISE', 'NEUTRAL']
model_input = np.expand_dims(normalized,0)
scores = model.predict(model_input).flatten()
scores
df = pd.DataFrame()
df["Emotion"] = EMOTIONS
df["Scores"] = scores
px.bar(df, x='Emotion', y='Scores', title="Model scores for each emotion")
prediction = EMOTIONS[scores.argmax()]
print(prediction)



Overwriting app.py


To start the website server run the next cell and follow the link!

**Note**: To edit the website you will need to stop the next cell, make the edits, and then run the next cell again!



In [ ]:
from pyngrok import ngrok
#Publish Web App (Run this again whenever you make changes)
public_url = ngrok.connect(port='80')
print (public_url)
! streamlit run --server.port 80 app.py

NgrokTunnel: "http://8e5f23ea3171.ngrok.io" -> "http://localhost:80"
2021-06-15 23:43:44.508 An update to the [server] config option section was detected. To have these changes be reflected, please restart streamlit.
2021-06-15 23:43:45.019711: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:80
  External URL: http://35.245.9.15:80

2021-06-15 23:43:46.831188: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-06-15 23:43:46.842854: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-06-15 23:43:46.842930: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (a13b43863dda): /proc/driver/nvidia/version does not exist

**Exercise**: Edit the webapp with a better title and header. 

**Exercise**: Checkout the documentation [here](https://docs.streamlit.io/en/stable/api.html#display-text) and take a look around. What do you think you'd want to add to the app?



## Step 2: Loading the model

Since the focus of this session is deployment we'll skip over training the model and just load one that we've already trained :)

Here is a reference on how to save and load sklearn and tensorflow models!

For sklearn:
```
from joblib import dump, load

# ====== Save model ========
dump(model, 'filename.joblib') 

# ====== Load model ========
clf = load('filename.joblib') 
```

For tensorflow: 
```
import tensorflow as tf

# ====== Save model ========
model.save("filename.h5")

# ====== Load model ========
tf.keras.models.load_model("filename.h5")
```

Our model today is going to be a tensorflow model!




In [14]:
model = tf.keras.models.load_model("emotion_detection_model_for_streamlit.h5")

In [8]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 1, 1, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 1024)              525312    
_________________________________________________________________
dropout_14 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_24 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_15 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 5)                

In [9]:
model.input_shape

(None, 48, 48, 3)

**Questions**: 

* Briefly discuss the model. What do you think the inputs to this model are like based on the input shape?

* How many emotions are we classifying to?

**Exercise**: Let's now try to load the model into the website. Write the model to the screen with the st.write function to make sure it worked :)

Note: You can either copy the %%writefile cell from the before or you can open the app.py file and directly edit it! 

Hint: If you're running into an import error, you need to include the import statement from within app.py!

## Step 3: Accept file uploads

We want to allow the user of the website to upload their own images to our website. This can be acomplished by the following line: 

`f = st.file_uploader("Upload Image")`

We can then extract and display with the following lines of code

```
if f is not None: 
  file_bytes = np.asarray(bytearray(f.read()), dtype=np.uint8)
  image = cv2.imdecode(file_bytes, 1)
  st.image(image, channels="BGR")
```

Exercise: Add this code to app.py, run the website and try it out!

Hint: Dont forget to 

`import numpy as np` 

and 

`import cv2` 

at the top of the file.

## Step 4: Preparing the input

Now that we can load an image, we just need to feed it through to the classifier. But there's something we missed - Our model takes in a very particular kind of image. 

**Quesiton:** What size of an image does our model take again?

Besides this, **we also have the requirement that the images are greyscaled and have values between 0 and 1.** 

Let's load a random image and check if it satisfies these properties.

In [56]:
with open("happy.png", "rb") as f:
  file_bytes = np.asarray(bytearray(f.read()))
  image = cv2.imdecode(file_bytes, 1)

print("Shape of image:")
print(image.shape)
print(f"Maximum value in the image: ")
print(image.max())
print("Minimum value in the image:")
print(image.min())

# Here we convert to RGB because our plotting function takes in RGB images
px.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

Shape of image:
(473, 457, 3)
Maximum value in the image: 
255
Minimum value in the image:
0


So it looks like there are 3 things to do: 
* Resize
* Normalize (get everything between 0-1)
* Greyscale

In [57]:
resized = cv2.resize(image, (48, 48), interpolation=cv2.INTER_LANCZOS4)
print(resized.shape)
px.imshow(cv2.cvtColor(resized, cv2.COLOR_BGR2RGB))

(48, 48, 3)


In [58]:
gray_1d = np.mean(resized, axis=-1)
gray = np.zeros_like(resized)
gray[:,:,0] = gray_1d
gray[:,:,1] = gray_1d
gray[:,:,2] = gray_1d

In [59]:
px.imshow(gray)

In [60]:
normalized = gray/255
px.imshow(normalized)

To check that everything is now good, let's print out the relevant stats again

In [61]:
print("Shape of image:")
print(normalized.shape)
print(f"Maximum value in the image: ")
print(normalized.max())
print("Minimum value in the image:")
print(normalized.min())

Shape of image:
(48, 48, 3)
Maximum value in the image: 
0.9568627450980393
Minimum value in the image:
0.03137254901960784


**Question:** Are we good?

## Step 5: Call the model on the prepared in put and write the results to the screen!

We can then get the predictions of our model with the following code: 

In [62]:
EMOTIONS = ['ANGRY', 'HAPPY', 'SAD', 'SURPRISE', 'NEUTRAL']

In [63]:
model_input = np.expand_dims(normalized,0)
scores = model.predict(model_input).flatten()
scores

array([0.69482195, 0.7817477 , 0.5981034 , 0.1195201 , 0.32736248],
      dtype=float32)

**Question:** What does the expand_dims function do to the shape of the image? Compare `normalized.shape` to `model_input.shape` to find out!

**Question**: 
* Why do you think this step is necessary?
* How do you interpret the output of this model?


Let's plot out the model output below.

In [64]:
  df = pd.DataFrame()
  df["Emotion"] = EMOTIONS
  df["Scores"] = scores
  px.bar(df, x='Emotion', y='Scores', title="Model scores for each emotion")

**Discuss:** How can we extract the predicted emotion based on the model output?

**Exercise:** Implement the strategy you just discussed!

In [65]:
prediction = EMOTIONS[scores.argmax()]
print(prediction)

HAPPY


**Exercise**: Now we're ready to add everything into the website. Here is the bare minimum that we need in our website:

* Copy the code that preprocesses the image so it can be read by our model. 
* Run the model on the input and display the prediction
* Also display the model's predicted score

**Extra credit**: 
* Include a graph of the model's predicted scores (Hint: `st.write`)
* Anything else you want to add - get creative! Check out the documentation [here](https://docs.streamlit.io/en/stable/api.html) for ideas and inspiriation. 


**NOTE**: You will likely run into bugs - this is a very normal part of programming. Check the error messages and try to figure out the problem. If you're running into a bug feel free to share it with the class so we can all work on it together!